In [1]:
import sys
import os
sys.path.append('../dags')

from database import create_database_engine


# 환경 변수 설정
os.environ["POSTGRES_USER"] = "airflow"
os.environ["POSTGRES_PASSWORD"] = "airflow"
os.environ["POSTGRES_DB"] = "events"
os.environ["POSTGRES_PORT"] = "5433"

engine = create_database_engine(host="127.0.0.1")
print("Successfully connected to the database")

Successfully connected to the database


In [2]:
import pandas as pd
from datetime import datetime

# 변수 설정
label_table = "kind"
return_table = "abnormal_return_kind_log"
volume_table = "abnormal_return_kind"   # volume은 이 테이블에서 select
label_col = "label"

# abnormal return 컬럼
abn_return_cols = (
    [f"abn_ret_minus_{i}m" for i in range(59, 0, -1)] +
    ["abn_ret_0m"] +
    [f"abn_ret_{i}m" for i in range(1, 61)]
)


# 포함할 cum_volume 컬럼
volume_cols = [
    "cum_volume_minus_60m", "cum_volume_minus_50m", "cum_volume_minus_40m", "cum_volume_minus_30m", "cum_volume_minus_20m", "cum_volume_minus_10m",
    "cum_volume_10m", "cum_volume_20m", "cum_volume_30m", "cum_volume_40m", "cum_volume_50m", "cum_volume_60m"
]

# SQL 컬럼 문자열 (event_ts는 return_table에서, abn_*은 return_table, volume_*은 volume_table, label은 kind에서)
sql_columns = (
    [f"ar.event_ts"] +
    [f"ar.{col}" for col in abn_return_cols] +
    [f"vol.{col}" for col in volume_cols] +
    [f"k.{label_col}"]
)
sql_columns_str = ",\n    ".join(sql_columns)

query = f"""
SELECT {sql_columns_str}
FROM {return_table} ar
JOIN {volume_table} vol ON ar.event_id = vol.event_id
JOIN "{label_table}" k ON ar.event_id = k.id
ORDER BY ar.event_ts ASC
"""

df_total = pd.read_sql(query, engine)
df_total["period_dummy"] = (
    pd.to_datetime(df_total["event_ts"], utc=True) > pd.Timestamp("2022-06-30", tz="UTC")
).astype(int)
df_total

DETAIL:  The database was created using collation version 2.36, but the operating system provides version 2.41.
HINT:  Rebuild all objects in this database that use the default collation and run ALTER DATABASE events REFRESH COLLATION VERSION, or build PostgreSQL with the right library version.


,event_ts,abn_ret_minus_59m,abn_ret_minus_58m,abn_ret_minus_57m,abn_ret_minus_56m,abn_ret_minus_55m,abn_ret_minus_54m,abn_ret_minus_53m,abn_ret_minus_52m,abn_ret_minus_51m,...,cum_volume_minus_20m,cum_volume_minus_10m,cum_volume_10m,cum_volume_20m,cum_volume_30m,cum_volume_40m,cum_volume_50m,cum_volume_60m,label,period_dummy
0,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,34208.0,34208.0,74063.0,77336.0,88710.0,95588.0,104470.0,106830.0,1,0
1,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37613.0,37413.0,58798.0,71353.0,81261.0,92625.0,102312.0,109293.0,0,0
2,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,37613.0,37413.0,58798.0,71353.0,81261.0,92625.0,102312.0,109293.0,0,0
3,2021-01-04 10:14:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,200844.0,186392.0,863840.0,993822.0,1195841.0,1288467.0,1319037.0,1389208.0,1,0
4,2021-01-04 10:16:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,204731.0,119572.0,59246.0,121680.0,167284.0,222801.0,257635.0,316213.0,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8651,2023-12-28 14:46:00+00:00,-0.000361,0.000000,-0.000946,0.000585,0.000361,-0.000721,0.001305,-0.002251,0.000361,...,5124.0,4403.0,115783.0,141291.0,152308.0,159431.0,159431.0,159431.0,1,1
8652,2023-12-28 14:53:00+00:00,-0.000883,0.000139,-0.003026,-0.000327,-0.000418,-0.001025,-0.000279,-0.000418,0.000139,...,17937.0,14085.0,17884.0,24052.0,34468.0,34468.0,34468.0,34468.0,1,1
8653,2023-12-28 14:56:00+00:00,0.000279,-0.000418,-0.000418,-0.000279,0.004040,0.000139,-0.002226,0.005926,-0.004580,...,5502.0,2973.0,2660.0,11246.0,23002.0,23002.0,23002.0,23002.0,0,1
8654,2023-12-28 14:56:00+00:00,0.000279,-0.000418,-0.000418,-0.000279,0.004040,0.000139,-0.002226,0.005926,-0.004580,...,5502.0,2973.0,2660.0,11246.0,23002.0,23002.0,23002.0,23002.0,0,1


In [3]:
for m in range(10, 61, 10):
    plus_cols = [
        f"abn_ret_minus_{abs(i)}m" if i < 0 else f"abn_ret_{i}m"
        for i in range(-m+1, m + 1)
    ]
    df_total[f"CAR_{m}m"] = df_total[plus_cols].sum(axis=1)

In [4]:
df_total

,event_ts,abn_ret_minus_59m,abn_ret_minus_58m,abn_ret_minus_57m,abn_ret_minus_56m,abn_ret_minus_55m,abn_ret_minus_54m,abn_ret_minus_53m,abn_ret_minus_52m,abn_ret_minus_51m,...,cum_volume_50m,cum_volume_60m,label,period_dummy,CAR_10m,CAR_20m,CAR_30m,CAR_40m,CAR_50m,CAR_60m
0,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,104470.0,106830.0,1,0,-0.004914,0.007855,-0.000312,0.000670,0.008463,0.004906
1,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,102312.0,109293.0,0,0,0.016961,0.023220,0.021990,0.022972,0.019986,0.023341
2,2021-01-04 10:08:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,102312.0,109293.0,0,0,0.016961,0.023220,0.021990,0.022972,0.019986,0.023341
3,2021-01-04 10:14:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1319037.0,1389208.0,1,0,0.037109,0.039253,0.046965,0.025485,0.029299,0.020446
4,2021-01-04 10:16:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,257635.0,316213.0,-1,0,-0.001911,-0.023724,-0.020824,-0.024954,-0.021079,-0.033610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8651,2023-12-28 14:46:00+00:00,-0.000361,0.000000,-0.000946,0.000585,0.000361,-0.000721,0.001305,-0.002251,0.000361,...,159431.0,159431.0,1,1,0.089213,0.067843,0.071069,0.063784,0.064817,0.063150
8652,2023-12-28 14:53:00+00:00,-0.000883,0.000139,-0.003026,-0.000327,-0.000418,-0.001025,-0.000279,-0.000418,0.000139,...,34468.0,34468.0,1,1,0.003182,-0.005646,-0.003085,-0.005461,-0.005361,-0.011459
8653,2023-12-28 14:56:00+00:00,0.000279,-0.000418,-0.000418,-0.000279,0.004040,0.000139,-0.002226,0.005926,-0.004580,...,23002.0,23002.0,0,1,0.002087,-0.007965,-0.008408,-0.020613,-0.025165,-0.022841
8654,2023-12-28 14:56:00+00:00,0.000279,-0.000418,-0.000418,-0.000279,0.004040,0.000139,-0.002226,0.005926,-0.004580,...,23002.0,23002.0,0,1,0.002087,-0.007965,-0.008408,-0.020613,-0.025165,-0.022841


In [5]:
df_total.filter(regex="^cum_").head()

,cum_volume_minus_60m,cum_volume_minus_50m,cum_volume_minus_40m,cum_volume_minus_30m,cum_volume_minus_20m,cum_volume_minus_10m,cum_volume_10m,cum_volume_20m,cum_volume_30m,cum_volume_40m,cum_volume_50m,cum_volume_60m
0,34208.0,34208.0,34208.0,34208.0,34208.0,34208.0,74063.0,77336.0,88710.0,95588.0,104470.0,106830.0
1,37913.0,37913.0,37913.0,37813.0,37613.0,37413.0,58798.0,71353.0,81261.0,92625.0,102312.0,109293.0
2,37913.0,37913.0,37913.0,37813.0,37613.0,37413.0,58798.0,71353.0,81261.0,92625.0,102312.0,109293.0
3,200844.0,200844.0,200844.0,200844.0,200844.0,186392.0,863840.0,993822.0,1195841.0,1288467.0,1319037.0,1389208.0
4,213376.0,213376.0,213217.0,209721.0,204731.0,119572.0,59246.0,121680.0,167284.0,222801.0,257635.0,316213.0


In [6]:
import numpy as np

# 10, 20, ..., 60 윈도우에 대해 delta_cum_volume 계산
for m in range(10, 61, 10):
    denom = 1 + df_total[f"cum_volume_minus_{m}m"]
    num = 1 + df_total[f"cum_volume_{m}m"]
    df_total[f"delta_cum_volume_{m}m"] = np.log(num / denom)

# min-max scaling for each delta_cum_volume_{m}m to [-1, 1]
for m in range(10, 61, 10):
    col = f"delta_cum_volume_{m}m"
    x = df_total[col].values
    x_min = np.nanmin(x)
    x_max = np.nanmax(x)
    df_total[col] = 2 * (x - x_min) / (x_max - x_min) - 1

# 누적 거래량으로 시작하는 컬럼 모두 drop
cum_cols_to_drop = [col for col in df_total.columns if col.startswith("cum_volume_")]
df_total = df_total.drop(columns=cum_cols_to_drop)
df_total.filter(regex="^delta_cum_").head()


,delta_cum_volume_10m,delta_cum_volume_20m,delta_cum_volume_30m,delta_cum_volume_40m,delta_cum_volume_50m,delta_cum_volume_60m
0,0.127802,0.166488,0.168029,0.097967,0.100136,0.075245
1,0.083598,0.144072,0.145074,0.082783,0.086337,0.066539
2,0.083598,0.144072,0.145074,0.082783,0.086337,0.066539
3,0.232819,0.266594,0.269576,0.191918,0.185550,0.161737
4,-0.075673,-0.004242,0.023991,-0.013224,-0.003374,-0.005831


In [7]:
df_total.filter(regex="^delta_cum_").describe()


,delta_cum_volume_10m,delta_cum_volume_20m,delta_cum_volume_30m,delta_cum_volume_40m,delta_cum_volume_50m,delta_cum_volume_60m
count,8656.000000,8656.000000,8656.000000,8656.000000,8656.000000,8656.000000
mean,0.114017,0.144022,0.123198,0.048590,0.037697,0.006608
std,0.149567,0.142230,0.139836,0.130971,0.134232,0.133269
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,0.037704,0.070676,0.050984,-0.022058,-0.034948,-0.064903
50%,0.103214,0.137011,0.118083,0.042241,0.030176,0.001142
75%,0.186815,0.216886,0.193538,0.114304,0.102081,0.070575
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
df_total[df_total["period_dummy"] == 1].head()

,event_ts,abn_ret_minus_59m,abn_ret_minus_58m,abn_ret_minus_57m,abn_ret_minus_56m,abn_ret_minus_55m,abn_ret_minus_54m,abn_ret_minus_53m,abn_ret_minus_52m,abn_ret_minus_51m,...,CAR_30m,CAR_40m,CAR_50m,CAR_60m,delta_cum_volume_10m,delta_cum_volume_20m,delta_cum_volume_30m,delta_cum_volume_40m,delta_cum_volume_50m,delta_cum_volume_60m
5072,2022-07-01 10:03:00+00:00,-0.007148,-0.005285,0.000471,-0.012082,0.008214,-0.002471,0.000944,-0.006312,0.002364,...,-0.003640,0.009501,-0.007204,-0.008546,-0.005757,0.083246,0.073672,0.028160,0.019277,-0.036871
5073,2022-07-01 10:24:00+00:00,0.000476,0.000476,-0.007229,-0.000477,-0.002379,-0.001425,-0.000949,0.008139,-0.000476,...,0.030922,0.029071,0.039132,0.031948,0.188400,0.321001,0.321234,0.214157,0.150370,0.115759
5074,2022-07-01 10:26:00+00:00,0.002865,-0.000477,-0.002379,0.000009,0.003340,0.002373,-0.000476,-0.002376,-0.001904,...,0.006729,0.024854,0.021497,0.021057,0.241785,0.170342,0.160364,0.060437,0.033450,-0.037571
5075,2022-07-01 10:43:00+00:00,-0.000162,-0.001130,-0.000968,0.000322,-0.000967,-0.000644,0.000161,-0.003180,0.000483,...,0.011305,0.009232,0.003966,0.000721,0.011265,0.040839,0.022614,-0.044183,-0.056170,-0.086890
5076,2022-07-01 11:03:00+00:00,0.000968,0.002677,-0.000773,-0.001581,-0.000484,-0.001068,0.000162,-0.001101,-0.001531,...,0.008496,0.010007,0.009466,0.007764,0.087719,0.028894,-0.021563,-0.086223,-0.073627,-0.101968


# 방향 정합성

### 📘 모형식 정의

$$
\text{logit}^{-1}(x) = \frac{1}{1 + e^{-x}}
$$

$$
Pr(hit_{i,t} = 1) = \text{logit}^{-1}(\alpha + \beta \cdot period\_dummy_i)
$$
---

### 📊 변수 설명


이벤트 *i*, 창 *t* 에 대해 Delta CAR, post CAR 2개의 지표를 사용

$$
t \in \{10,\,20,\,\dots,\,60\}
$$

$$
\Delta CAR_{i,t} = CAR_{post,i,t} - CAR_{pre,i,t}  
$$

$$
CAR_{post,i,t}
$$

$$
label\_sign_i =
\begin{cases}
+1, & \text{긍정 (1)} \\
0, & \text{중립 (0)} \\
-1, & \text{부정 (-1)} 
\end{cases}
$$



$$
hit_{i,t} =
\begin{cases}
1, & \text{if } \operatorname{sign}(\Delta CAR_{i,t}) = label\_sign_i \\
0, & \text{otherwise}
\end{cases}
$$

---

### 📑 가설 설정

$$
\begin{aligned}
H_0 &: \beta = 0 \quad &(\text{도입 전후 차이 없음}) \\
H_1 &: \beta > 0 \quad &(\text{도입 후 정합률 상승})
\end{aligned}
$$

---

### 📈 해석

- delta CAR: β > 0 이고 **통계적으로 유의**하다면  → “텔레그램 서비스 출시 이후 과잉반응 감소 / 즉시 반영 후 빠른 수렴 → 정보의 선반영 및 사전 확산이 강화 → 정보 효율성 향상”을 의미한다고 볼 수 있다.
- Post CAR: β > 0 이고 **통계적으로 유의**하다면  → “텔레그램 서비스 출시 이후 이벤트 발표 이후의 정합률(정확도) 증가 → 시장 반응의 즉시 반영 강화 → 정보 효율성 향상”을 의미한다고 해석할 수 있다.


“중립 이벤트를 포함한 모델에서는 텔레그램 도입 이후 정합률이 유의하게 상승하였으나,
중립을 제외한 경우에는 그 효과가 약화되었다. 이는 정보 효율성 개선이
주로 가격 반응의 ‘적정성(overshooting 감소)’ 혹은 ‘비과잉 반응’ 영역에서 나타났음을 시사한다.
다시 말해, 시장이 새로운 정보를 과도하게 반응하지 않고 신속히 흡수하게 된 결과로 볼 수 있다.”

또는 요약형으로:

“효율성 개선은 방향성 예측력이 아니라, 불필요한 반응을 줄이는 형태로 나타났다.”

## 중립 포함

In [9]:
import sys
import os
sys.path.append('.')

from hypothesis_test_volume import (
    print_sample_summary_with_neutral,
    logistic_hit_delta_with_neutral_and_vol_log,
    run_logistic_table
)

# === 전체 파이프라인 (중립 포함, 중립은 ΔCAR/POSTCAR 절대값 기준) ===

df = df_total.copy()
df["label_sign"] = df["label"]

df_full = print_sample_summary_with_neutral(df, "label_sign")
# 창 목록
windows = list(range(10, 61, 10))

# ΔCAR + 거래량 공변량 (중립 포함)
results_delta_w_neutral_vol = run_logistic_table(
    logistic_hit_delta_with_neutral_and_vol_log,
    df_full,
    windows,
    label="ΔCAR + volume (neutral incl.)",
    covar_prefix="delta_cum_volume_",  # 기본값과 동일
    add_interaction=False,             # 상호작용 보려면 True
    robust_se=True
)



=== Sample summary (including neutral) ===
Total events:        8,656
Neutrals:            2,980
  - Positive (1):    4,906
  - Negative (-1):   770


--- ΔCAR + volume (neutral incl.) 기준 결과 ---
 t beta_star    std  t_stat  p_value  odds_ratio  p_before  p_after  diff_pp  n_obs     R2  adj_R2  neutral_epsilon gamma_vol  std_vol    p_vol
10 0.1551*** 0.0458   3.382 0.000720       1.168     0.630    0.665     3.54   8656 0.0012  0.0006         0.006245   -0.2089   0.1524 0.170596
20  0.1321** 0.0454   2.912 0.003595       1.141     0.616    0.647     3.07   8656 0.0008  0.0002         0.007062   -0.0909   0.1589 0.567302
30   0.1014* 0.0451   2.246 0.024696       1.107     0.611    0.635     2.38   8656 0.0005 -0.0000         0.007506    0.1383   0.1599 0.387357
40    0.0486 0.0452   1.077 0.281546       1.050     0.619    0.630     1.14   8656 0.0001 -0.0004         0.008119   -0.0677   0.1691 0.688893
50    0.0835 0.0449   1.858 0.063105       1.087     0.604    0.624     1.98   8656 0

## 중립 제거

In [10]:
from hypothesis_test_volume import (
    print_sample_summary,
    logistic_hit_delta_with_vol_log,
    run_logistic_table
)

# === 전체 파이프라인 ===

df = df_total.copy()
df["label_sign"] = df["label"]
df_nn = print_sample_summary(df, "label_sign")
windows = list(range(10, 61, 10))

# ΔCAR + volume (중립 제거)
results_delta_vol = run_logistic_table(
    logistic_hit_delta_with_vol_log,
    df_nn, windows,
    label="ΔCAR + volume (neutral removed)",
    covar_prefix="delta_cum_volume_",  # 이미 만드신 컬럼 접두사
    add_interaction=False,             # 필요 시 True
    robust_se=True
)



=== Sample summary (neutral removed) ===
Total events:        8,656
Removed neutrals:    2,980
Used (non-neutral):  5,676
  - Positive (1):    4,906
  - Negative (-1):   770


--- ΔCAR + volume (neutral removed) 기준 결과 ---
 t beta_star    std  t_stat  p_value  odds_ratio  p_before  p_after  diff_pp  n_obs     R2  adj_R2 gamma_vol  std_vol  p_vol
10    0.0484 0.0600   0.807 0.419770       1.050     0.714    0.723     0.96   5676 0.0164  0.0155 2.1347***   0.2260    0.0
20    0.0294 0.0586   0.501 0.616030       1.030     0.691    0.697     0.61   5676 0.0166  0.0157 2.2329***   0.2196    0.0
30    0.0225 0.0583   0.386 0.699142       1.023     0.684    0.689     0.47   5676 0.0179  0.0171 2.3466***   0.2229    0.0
40   -0.0624 0.0585  -1.067 0.285893       0.940     0.698    0.685    -1.30   5676 0.0170  0.0161 2.4489***   0.2407    0.0
50   -0.0262 0.0579  -0.453 0.650413       0.974     0.679    0.674    -0.56   5676 0.0196  0.0187 2.5842***   0.2347    0.0
60   -0.0578 0.0577  -1.003 